In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.colors
import mplotutils as mpu
import cartopy.crs as ccrs
from netCDF4 import Dataset
import metpy.calc as mpcalc
from dypy.lagranto import Tra
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.feature as cfeature
import matplotlib.patheffects as path_effects
from wrf import getvar, to_np, latlon_coords, CoordPair, interplevel

%matplotlib inline

# Map of Terrain Height

In [ ]:
# Preprocessing of TITAN data for thunderstorm initiation locations
titan_data = '/scratch3/tim/wrf_4.0.1/P3/tracks/tracks_20180531.csv.gz'
titan_data_CC = '/scratch3/tim/wrf_4.0/P3_CC/tracks/tracks_20180531.csv.gz'
df = pd.read_csv(titan_data_CC, sep=',', header=41)

# Selcting storms lasting longer than 10 min (=600s)
df_duration_600 = df[df['stormDuration'] > 600]

# Reset index
df_duration_600 = df_duration_600.reset_index()

# Selcting storms initiated over Switzerland (5.5, 11, 45.5, 48)
df_ch_min = df_duration_600[(df_duration_600['xCoord'] > 5.9) & (df_duration_600['yCoord'] > 45.75)]
df_ch_max = df_ch_min[(df_ch_min['xCoord'] < 10.5) & (df_ch_min['yCoord'] < 47.85)]

df_initiations = df_ch_max[df_ch_max['originTime'] == df_ch_max['timestamp'] ]

# Reset index
df_initiations = df_initiations.reset_index()

# Create file with initiation information
initiations = df_initiations.filter(['stormID', 'stormDuration', 'xCoord', 'yCoord', 'originTime', 
                                     'timestamp',], axis='columns')

#initiations_2018 = initiations

In [ ]:
def terrain_height_map(case_study_name, subset=False, rectangle=False, subset_small=False,
                        initiation=False, cross_section=False, trajs_start=False, divergence=False,
                        case_studies=False, title=False, initiations_titan=False, save=False):
    '''This function plots the terrain height of the 
    investigation area with possible add-on features: subset (smaller extent),
    rectangle of subset and initiation location.'''

    # Predefine some variables
    variable_name = 'HGT'
    title_name = 'Terrain Height of Investigation Area'
    colorbar_label = 'Terrain Height [$km$]'

    # Define Location of initiation
    case_study1 = [7.56971, 47.4961]
    case_study2 = [7.63891, 47.0546]
    case_study3 = [8.64449, 47.5522]
    case_study4 = [8.67235, 47.629]
    case_study5 = [6.95974, 46.4482]
    
    #initiation_location = case_study4
    if case_study_name == 'case_study_1':
        initiation_location =  [7.56971, 47.4961]
        case_study_abbr = 'cs1'
        start_time = '1630'
        end_time = '0800'
        date = '2018-05-22_17:00'
        
    elif case_study_name == 'case_study_2':
        initiation_location =  [7.63891, 47.0546]
        case_study_abbr = 'cs2'
        start_time = '1405'
        end_time = '0600'
        date = '2018-05-09_14:05'
        
    elif case_study_name == 'case_study_3':
        initiation_location =  [8.64449, 47.5522]
        case_study_abbr = 'cs3'
        start_time = '2015'
        end_time = '1200'
        date = '2018-05-12_20:15'
    
    elif case_study_name == 'case_study_4':
        initiation_location =  [8.67235, 47.629]
        case_study_abbr = 'cs4'
        start_time = '0100'
        end_time = '1700'
        date = '2018-05-10_01:00'
        
    elif case_study_name == 'case_study_5':
        initiation_location =  [6.95974, 46.4482]
        case_study_abbr = 'cs5'
        start_time = '1450'
        end_time = '0600'
        date = '2018-05-30_14:50'
        
    # Change extent of plot
    initiation_area_extent = [5.9, 10.5, 45.75, 47.85]
    subset_extent = [initiation_location[0]-1, initiation_location[0]+1,
                    initiation_location[1]-0.5, initiation_location[1]+0.5]

    subset_extent = initiation_area_extent

    ##################################################
    # Cut terrain
    class MidpointNormalize(matplotlib.colors.Normalize):
        def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
            self.vcenter = vcenter
            matplotlib.colors.Normalize.__init__(self, vmin, vmax, clip)

        def __call__(self, value, clip=None):
            x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
            return np.ma.masked_array(np.interp(value, x, y))

    midnorm = MidpointNormalize(vmin=-0.5, vcenter=0, vmax=4)
    
    colors_undersea = plt.cm.terrain(np.linspace(0.11, 0.13, 256))
    colors_land = plt.cm.terrain(np.linspace(0.23, 1, 256))
    all_colors = np.vstack((colors_undersea, colors_land))
    terrain_map = matplotlib.colors.LinearSegmentedColormap.from_list('terrain_map',
            all_colors)
    
    
    pseudo_ocean_color = plt.cm.terrain(np.linspace(0.23, 0.25, 200))
    pseudo_ocean_colormap = matplotlib.colors.LinearSegmentedColormap.from_list('cut_terrain', 
                                                                                pseudo_ocean_color)
    
    ##################################################
    
    # Load the netCDF file
    ncfile = Dataset('/scratch3/thomasl/work/data/case_study_4/' \
                      'wrfout_d02_2018-05-10_00:55:00')
    # Load Variable
    variable = getvar(ncfile, variable_name)/1000
    variable_nometa = getvar(ncfile, variable_name, meta=False)/1000

    # Define cart projection
    lats, lons = latlon_coords(variable)

    cart_proj = ccrs.LambertConformal(central_longitude=8.722206, 
                                        central_latitude=46.73585)

    # Create figure
    fig = plt.figure(figsize=(15,10))
    ax = plt.axes(projection=cart_proj)
    
    # Set map bounds
    domain_extent = [3.701088, 13.814863, 43.85472,49.49499]
    
    if subset == True:
        if divergence == True:
                subset_extent = [initiation_location[0]-1, initiation_location[0]+1,
                                    initiation_location[1]-0.5, initiation_location[1]+0.5] 
                ax.set_extent([subset_extent[0],subset_extent[1],
                                       subset_extent[2],subset_extent[3]],
                                         ccrs.PlateCarree())
        else:
            ax.set_extent([subset_extent[0],subset_extent[1],
                               subset_extent[2],subset_extent[3]],
                                 ccrs.PlateCarree())
                
    else:
        ax.set_extent([domain_extent[0]+0.7,domain_extent[1]-0.7,
                   domain_extent[2]+0.2,domain_extent[3]-0.2],
                     ccrs.PlateCarree())

    # Plot the variable with colorbar
    levels = np.arange(0, variable_nometa.max(), 0.2)
    
    if initiations_titan == True or divergence == True:
        cmap = get_cmap('Greys')
    else:
        cmap = terrain_map
    if divergence == True:
            height_plot = ax.contour(to_np(lons), to_np(lats), to_np(variable), 
                        norm=midnorm, extend='both', levels=levels, linewidths=1.5,
                        transform=ccrs.PlateCarree(), zorder=7, 
                        colors='k', alpha=0.5)
            clabels = height_plot.clabel(inline=True, fontsize=12.5, fmt='%i')
            for clabel in clabels: 
                clabel.set_zorder(12)
            
    else:
        height_plot = plt.contourf(to_np(lons), to_np(lats), to_np(variable), 
                            norm=midnorm, extend='both', levels=levels,
                            transform=ccrs.PlateCarree(), 
                            cmap=cmap, alpha=1)
    
    if divergence == True:
        variable_name = 'ua'
        colorbar_label_var = 'Divergence [$10^{-3}$ $s^{-1}$]'
        save_name = 'divergence'
        if case_study_name == 'case_study_4':
            variable_min = -1
            variable_max = 1         
        elif  case_study_name == 'case_study_3':
            variable_min = -2.5
            variable_max = 2.5 
        elif  case_study_name == 'case_study_1':
            variable_min = -1
            variable_max = 1  
        elif  case_study_name == 'case_study_2':
            variable_min = -1
            variable_max = 1 
        elif  case_study_name == 'case_study_5':
            variable_min = -2.5
            variable_max = 2.5 
        
        ncfile_div = Dataset('/scratch3/thomasl/work/data/{}/' \
                      'wrfout_d02_{}:00'.format(case_study_name, date))
        
        variable = getvar(ncfile_div, variable_name)
        p = getvar(ncfile_div, 'pressure')
        pressure_level = 900
        
        variable_pressure = interplevel(variable, p, 
                                        pressure_level)
        variable = variable_pressure
        
        va = getvar(ncfile_div, 'va')

        p = getvar(ncfile_div, 'pressure')
 
        levels_num = 11
        levels = np.linspace(variable_min, variable_max, levels_num)
            
        v_pressure = interplevel(va, p, pressure_level)

        u_wind = variable.squeeze()
        v_wind = v_pressure.squeeze()

        u_wind.attrs['units']='meters/second'
        v_wind.attrs['units']='meters/second'

        lats, lons = latlon_coords(variable)
        lats = lats.squeeze()
        lons = lons.squeeze()

        dx, dy = mpcalc.lat_lon_grid_deltas(to_np(lons), to_np(lats))

        div = mpcalc.divergence(u_wind, v_wind, dx, dy, dim_order='yx')
        div = div*1e3
                
        cmap = plt.get_cmap('PRGn')

        variable_plot = plt.contourf(to_np(lons), to_np(lats), div, 
                         levels=levels, transform=ccrs.PlateCarree(), zorder=6, 
                         cmap=cmap, extend='both')#, norm=new_norm)
        initiation_color = 'k*'
        cbar_var = mpu.colorbar(variable_plot, ax, extend='both', orientation='vertical', 
                    aspect=20, shrink=.05)  
        cbar_var.set_label(colorbar_label_var, fontsize=20)
        cbar_var.set_ticks(levels)
        cbar_var.ax.tick_params(labelsize=15)        

    # Plot pseudo ocean
    data = xr.open_dataset('/scratch3/thomasl/work/data/case_study_4/' \
                      'wrfout_d02_2018-05-10_00:55:00')

    terrain = data.HGT.squeeze()
    pseudo_ocean = terrain.where(terrain < 0)

#    plt.contourf(lons, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                 transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.1, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                 transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.1, lats-0.05, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                 transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.2, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.2, lats-0.05, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.3, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.3, lats-0.05, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.05, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.05, lats-0.05, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.001, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.01, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.02, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.03, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.04, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons-0.001, lats-0.05, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons+0.01, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons+0.02, lats, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons, lats+0.05, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#    plt.contourf(lons, lats+0.06, pseudo_ocean, cmap=pseudo_ocean_colormap, 
#                transform=ccrs.PlateCarree(), zorder=2)
#
    if divergence == True:
        pass
    else:
        cbar = mpu.colorbar(height_plot, ax, extend='both', orientation='vertical', 
                            aspect=20, shrink=.05)  
        cbar.set_label(colorbar_label, fontsize=20)
        cbar.ax.tick_params(labelsize=15)        


    # Add borders, coastlines, lakes and rivers
    if initiations_titan == True:
        border_color = 'w'
    else:
        border_color = 'k'        
    ax.add_feature(cfeature.BORDERS.with_scale('10m'), zorder=5, 
                   linewidth=1.5, edgecolor=border_color)
    ax.add_feature(cfeature.COASTLINE.with_scale('10m'), zorder=5,
                  linewidth=1.5)
    if initiations_titan == True:
        pass
    else:
        ax.add_feature(cfeature.LAKES.with_scale('10m'), zorder=4)
        ax.add_feature(cfeature.RIVERS.with_scale('10m'), zorder=4)

    # Add rectangle for subset
    if rectangle == True:
        subset_extent = [initiation_location[0]-1, initiation_location[0]+1,
                        initiation_location[1]-0.5, initiation_location[1]+0.5]
        x = subset_extent[0]
        y = subset_extent[2]
        if case_studies == True:
            width = 4.6
            height = 2.1
        elif subset_small == True:
            x = subset_extent[0]+0.8
            y = subset_extent[2]+0.4
            width = 0.4
            height = 0.2

        else:
            width = 2
            height = 1
        rect = plt.Rectangle((x, y), width, height,
                             color='r', linewidth=2, fill=False, 
                              zorder=7, transform=ccrs.PlateCarree(),
                            path_effects=[path_effects.Stroke(linewidth=3, foreground='white'),
                       path_effects.Normal()])
        ax.add_patch(rect) 

    # Add initiation location of the thunderstorm
    if initiation == True:
        ax.plot(initiation_location[0], initiation_location[1], 'r*', 
                markersize=15, transform=ccrs.PlateCarree(), zorder=8,
               path_effects=[path_effects.Stroke(linewidth=3, foreground='white'),
                       path_effects.Normal()])
    
    # Add line for cross section
    if cross_section == True:
        ax.plot([initiation_location[0], initiation_location[0]],
                [initiation_location[1]-0.5, initiation_location[1]+0.5], color='r',
                transform=ccrs.PlateCarree(), zorder=7,
               path_effects=[path_effects.Stroke(linewidth=5, 
                            foreground='white'),path_effects.Normal()])
        
        ax.text(initiation_location[0]-0.08, (initiation_location[1]-0.5)-0.175, 'A', zorder=7, 
                      fontsize=15, weight='bold', transform=ccrs.PlateCarree(),
                      path_effects=[path_effects.Stroke(linewidth=3, foreground='white'),
                       path_effects.Normal()])
    
        ax.text(initiation_location[0]-0.08, (initiation_location[1]+0.5)+0.06, 'B', zorder=7, fontsize=15, 
                      weight='bold', transform=ccrs.PlateCarree(),
                      path_effects=[path_effects.Stroke(linewidth=3, foreground='white'),
                       path_effects.Normal()])
        
    if case_studies == True:
        ax.plot(case_study1[0], case_study1[1], 'r*', zorder=7, 
                transform=ccrs.PlateCarree(), markersize=15,
                path_effects=[path_effects.Stroke(linewidth=3, foreground='white'),
                       path_effects.Normal()])
        ax.text(case_study1[0]+0.075, case_study1[1], '$\\bf{Case\ Study\ I}$', 
                zorder=7, transform=ccrs.PlateCarree(), fontsize=20, color='r',
               path_effects=[path_effects.Stroke(linewidth=1.5, foreground='white'),
                       path_effects.Normal()])

        ax.plot(case_study2[0], case_study2[1], 'r*', zorder=7, 
                transform=ccrs.PlateCarree(), markersize=15,
                path_effects=[path_effects.Stroke(linewidth=3, foreground='white'),
                       path_effects.Normal()])
        ax.text(case_study2[0]+0.075, case_study2[1], '$\\bf{Case\ Study\ II}$', 
                zorder=7, transform=ccrs.PlateCarree(), fontsize=20, color='r',
                path_effects=[path_effects.Stroke(linewidth=1.5, foreground='white'),
                       path_effects.Normal()])

        ax.plot(case_study3[0], case_study3[1], 'r*', zorder=7, 
                transform=ccrs.PlateCarree(), markersize=15,
                path_effects=[path_effects.Stroke(linewidth=3, foreground='white'),
                       path_effects.Normal()])
        ax.text(case_study3[0]+0.075, case_study3[1]-0.075, '$\\bf{Case\ Study\ III}$', 
                zorder=7, transform=ccrs.PlateCarree(), fontsize=20, color='r',
                path_effects=[path_effects.Stroke(linewidth=1.5, foreground='white'),
                       path_effects.Normal()])

        ax.plot(case_study4[0], case_study4[1], 'r*', zorder=7, 
                transform=ccrs.PlateCarree(), markersize=15,
                path_effects=[path_effects.Stroke(linewidth=3, foreground='white'),
                       path_effects.Normal()])
        ax.text(case_study4[0]+0.075, case_study4[1], '$\\bf{Case\ Study\ IV}$', 
                zorder=7, transform=ccrs.PlateCarree(), fontsize=20, color='r',
                path_effects=[path_effects.Stroke(linewidth=1.5, foreground='white'),
                       path_effects.Normal()])

        ax.plot(case_study5[0], case_study5[1], 'r*', zorder=7, 
                transform=ccrs.PlateCarree(), markersize=15,
                path_effects=[path_effects.Stroke(linewidth=3, foreground='white'),
                       path_effects.Normal()])
        ax.text(case_study5[0]+0.075, case_study5[1], '$\\bf{Case\ Study\ V}$', 
                zorder=7, transform=ccrs.PlateCarree(), fontsize=20, color='r',
                path_effects=[path_effects.Stroke(linewidth=1.5, foreground='white'),
                       path_effects.Normal()])

        
    if trajs_start == True:
        traj_data_dir = '/scratch3/thomasl/work/retrospective_part/lagranto/' \
                        'traj_{}_{}_{}.ll'.format(case_study_abbr, start_time, end_time)        
        trajs = Tra()
        trajs.load_ascii(traj_data_dir)
        for t in trajs:
            ax.plot(t['lon'][0],t['lat'][0],'kx', markersize=15, zorder=6,
                    transform=ccrs.PlateCarree())   
            
    if initiations_titan == True:
        for i in initiations_2018.iterrows():
            ax.plot(i[1].xCoord, i[1].yCoord,'b+', markersize=10, zorder=6,
                    transform=ccrs.PlateCarree())
        for i in initiations.iterrows():
            ax.plot(i[1].xCoord, i[1].yCoord,'rx', markersize=10, zorder=6,
                    transform=ccrs.PlateCarree())
            
    # Add gridlines
    lon = np.arange(0, 20, 1)
    lat = np.arange(40, 60, 1)

    gl = ax.gridlines(xlocs=lon, ylocs=lat, zorder=6)

    # Add ticks and tick labels
    mpu.yticklabels(lat, ax=ax, fontsize=15)
    mpu.xticklabels(lon, ax=ax, fontsize=15)

    # Set title
    if title == True:
        if case_studies == True:
            ax.set_title('Case Study Locations with Terrain Height', fontsize=20)
        
        elif initiations_titan == True:
            ax.set_title('TITAN Initiation Locations with Terrain Height', fontsize=20)   
        elif divergence == True:
            ax.set_title('Horizontal Wind Divergence with Terrain Height', fontsize=20)   

        else:
            ax.set_title(title_name, fontsize=20)
            
    plt.show()
    
    if save == True:
        if rectangle == True:
            if subset_small == True:
                  fig.savefig('/scratch3/thomasl/work/terrain_plots/terrain_height_map_' \
                              '{}_subset_small.png'.format(case_study_name), 
                                bbox_inches='tight', dpi=300)
            else:              
                fig.savefig('/scratch3/thomasl/work/terrain_plots/terrain_height_map_' \
                          '{}_rectangle.png'.format(case_study_name), 
                            bbox_inches='tight', dpi=300)  
        elif cross_section == True:
              fig.savefig('/scratch3/thomasl/work/terrain_plots/terrain_height_map_' \
                          '{}_cross.png'.format(case_study_name), 
                            bbox_inches='tight', dpi=300)
        elif case_studies == True:
              fig.savefig('/scratch3/thomasl/work/terrain_plots/terrain_height_map_' \
                          '{}_case_studies.png'.format(case_study_name), 
                            bbox_inches='tight', dpi=300)
        elif initiations_titan == True:
              fig.savefig('/scratch3/thomasl/work/terrain_plots/terrain_height_map_' \
                          '{}_titan_initiations.png'.format(case_study_name), 
                            bbox_inches='tight', dpi=300)
        elif trajs_start == True:
              fig.savefig('/scratch3/thomasl/work/terrain_plots/terrain_height_map_' \
                          '{}_trajs_start.png'.format(case_study_name), 
                            bbox_inches='tight', dpi=300)
        elif divergence == True:
              fig.savefig('/scratch3/thomasl/work/terrain_plots/terrain_height_map_' \
                          '{}_divergence.png'.format(case_study_name), 
                            bbox_inches='tight', dpi=300)            
        elif subset == True:
            if subset_small == True:
                  fig.savefig('/scratch3/thomasl/work/terrain_plots/terrain_height_map_' \
                              '{}_subset_small.png'.format(case_study_name), 
                                bbox_inches='tight', dpi=300)
            else:
                  fig.savefig('/scratch3/thomasl/work/terrain_plots/terrain_height_map_' \
                              '{}_subset.png'.format(case_study_name), 
                                bbox_inches='tight', dpi=300)                
        else:
            fig.savefig('/scratch3/thomasl/work/terrain_plots/terrain_height_map_' \
                        '{}.png'.format(case_study_name), bbox_inches='tight', dpi=300)
            

In [ ]:
terrain_height_map('case_study_2', subset=True, rectangle=False, subset_small=False,
                  initiation=True, cross_section=False, trajs_start=False, divergence=True,
                   case_studies=False, title=False, initiations_titan=False, save=True)